In [60]:
import numpy as np
from scipy import signal
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.preprocessing import LabelBinarizer
import csv
import pickle
import datetime
from sklearn.preprocessing import MinMaxScaler

In [3]:
def data_preposcessing(x,y,chunk,fs,band,nothval,samplesize,channel_num=8,band2=None):
    #Divide datas to chunks, each chunk coorespond to specific channel with specific label.
    #Total chunk number = channel_num * record_size(chunk) *3
    _total_chunk=channel_num*chunk*3

    for k in range(channel_num):
        locals()["channel"+str(k+1)]=x[:,k]

    eventsplit=[]
    for u in range(int(chunk*3)):
        event_locator=u*samplesize
        eventsplit.append(int(event_locator))

    for i in np.unique(y):
        i=int(i)
        locals()['ind'+str(i+1)]=[]
    for split in eventsplit:
        label_finder=int(y[split])
        locals()['ind'+str(label_finder+1)].append([split,int(split+samplesize-1)])
    for i in np.unique(y):
        i=int(i)
    #     locals()['ind_'+str(i+1)]=np.where(y==i)[0] ##left=-1,(ind0) ; right=1,(ind2); focus=0,(ind1);

    #     locals()['ind'+str(i+1)]=[locals()['ind_'+str(int(i+1))][0]]  
    #     for j in range(len(locals()['ind_'+str(i+1)])-1):  
    #         if (locals()['ind_'+str(i+1)])[j]+1!=(locals()['ind_'+str(i+1)])[j+1]:
    #             locals()['ind'+str(i+1)].append((locals()['ind_'+str(i+1)])[j])
    #         if j>1 and (locals()['ind_'+str(i+1)])[j]-1!=(locals()['ind_'+str(i+1)])[j-1]:
    #             (locals()['ind'+str(i+1)]).append((locals()['ind_'+str(i+1)])[j])
    #     (locals()['ind'+str(i+1)]).append((locals()['ind_'+str(i+1)])[-1])   #find boundry of events

        if chunk!=len((locals()['ind'+str(i+1)])):
            print("error: chunk size and interval limit does not match. flag number:",i)
        for k in range(channel_num):
            for g in range(int(len((locals()['ind'+str(i+1)])))):
                locals()['channel'+str(k+1)+'_'+str(i+1)+'_chunk'+str(g+1)]=locals()['channel'+str(k+1)][(locals()['ind'+str(i+1)])[g][0]:(locals()['ind'+str(i+1)])[g][1]+1]

    total_chunks=0
    #apply bp and notch filter
    #variable_channel#_label#_chunk#

    for i in np.unique(y):
        i=int(i)
        for k in range(channel_num):
            for g in range(int(len((locals()['ind'+str(i+1)])))):
                data=locals()['channel'+str(k+1)+'_'+str(i+1)+'_chunk'+str(g+1)]
                freq,ffty=fft(data,fs)
                freq_ind=np.where((freq>=band[0])&(freq<=band[1]))[0]
                desired_freq=freq[freq_ind]
                amp_of_desired_freq=ffty[freq_ind]
                average_amp=np.mean(amp_of_desired_freq)
                if band2:
                    freq_ind2=np.where((freq>=band2[0])&(freq<=band2[1]))[0]
                    desired_freq2=freq[freq_ind2]
                    amp_of_desired_freq2=ffty[freq_ind2]
                    average_amp2=np.mean(amp_of_desired_freq2)
                locals()['average_amp_'+str(k+1)+'_'+str(i+1)+'_'+str(g+1)]=np.array([average_amp,average_amp2])
#                 bp_filtered=bandpass(band[0],band[1],data,fs) #apply band pass filter
#                 channel_filtered=notch(notchval,bp_filtered,fs)
#                 locals()['bp_filtered'+str(k+1)+'_'+str(i+1)+'_'+str(g+1)]=channel_filtered
                total_chunks+=1
    if total_chunks != _total_chunk: print('error: total chunks number does not match with the data acqusition')
    ##################CNN input matrix generation#################################
    labels=[]
    for i in np.unique(y): 
        i=int(i)
        for g in range(int(len((locals()['ind'+str(i+1)])))):
            for k in range(channel_num):
                data=locals()['average_amp_'+str(k+1)+'_'+str(i+1)+'_'+str(g+1)]
                if k==0:
                    CNNmatrices_mergechn=[data]
                else:
                    CNNmatrices_mergechn=np.append(CNNmatrices_mergechn,[data],axis=0)
            if g==0 and i==-1: CNNmatrices=CNNmatrices_mergechn.reshape(1,CNNmatrices_mergechn.shape[0],CNNmatrices_mergechn.shape[1])
            else: CNNmatrices=np.append(CNNmatrices,CNNmatrices_mergechn.reshape(1,CNNmatrices_mergechn.shape[0],CNNmatrices_mergechn.shape[1]),axis=0)
            labels=np.append(labels,[i])
    #     _shape=CNNmatrices.shape
    #     CNN_input=CNNmatrices.reshape(_shape[0],1,_shape[1],_shape[2])
    lb = LabelBinarizer()
    label_enc=lb.fit_transform(labels)
    return CNNmatrices,labels

In [212]:
# set path to datasets
datapath = '/home/jingyan/Documents/ME499-WinterProject/mindwave/src/motor_cortex_ml/data/'
def load_record(sec,runnum=7):
    for secnum in sec:
        for run in range(1,runnum): #6runs for each session
            savetag='_exp13_sec'+str(secnum)+'_run'+str(run)
            csvname =datapath + 'record'+savetag+'.csv'
            data = np.loadtxt(csvname,delimiter = ',')
            if run==1 and secnum==sec[0]:
                x = data[:,:-1]
                y = data[:,-1]
            else:
                x=np.append(x,data[:,:-1],axis=0)
                y=np.append(y,data[:,-1],axis=0)
    return x, y
def bandpass(start,stop,data,fs):
    bp_Hz = np.array([start, stop])
    b, a = signal.butter(5, bp_Hz / (fs *0.5), btype='bandpass')
    return signal.lfilter(b, a, data, axis=0)

def notch(val, data, fs):
    bp_stop_Hz = val + 3.0 * np.array([-1, 1])
    b, a = signal.butter(3, bp_stop_Hz / (fs *0.5), 'bandstop')
    return signal.lfilter(b, a, data)

def fft(data, fs):
    L = len(data)
    freq = np.linspace(0.0, 1.0 / (2.0 * fs **-1), int(L / 2))
    yi = np.fft.fft(data)#[1:]
    y = yi[range(int(L / 2))]
    # ysample= (2.0*abs(yi/L))[range(int(L/2))]
    return freq, abs(y)

channel_num=8
chunk=60 #number of trails for each task in one session
fs=250
band=(7,13)
band2=(18,30)
notchval=60
samplesize=7.5*250
train_data,train_label=load_record([5],4)
# test_data,test_label=load_record([4],3)
# train_data,train_labels=data_preposcessing(train_data,train_labels,chunk,fs,band,notchval,samplesize)

In [ ]:
for k in range(channel_num):
    locals()["channel"+str(k+1)]=x[:,k]
#Divide datas to chunks, each chunk coorespond to specific channel with specific y value.
#Total chunk number = channel_num * record_size(chunk) *4
_total_chunk=channel_num*chunk*4
for i in np.unique(y):
    i=int(i)
    locals()['ind_'+str(i+1)]=np.where(y==i)[0] ##left=-1,(ind0) ; right=1,(ind2); focus=0,(ind1);
   
    locals()['ind'+str(i+1)]=[locals()['ind_'+str(int(i+1))][0]]
    for j in range(len(locals()['ind_'+str(i+1)])-1):  
        if (locals()['ind_'+str(i+1)])[j]+1!=(locals()['ind_'+str(i+1)])[j+1]:
            locals()['ind'+str(i+1)].append((locals()['ind_'+str(i+1)])[j])
        if j>1 and (locals()['ind_'+str(i+1)])[j]-1!=(locals()['ind_'+str(i+1)])[j-1]:
            (locals()['ind'+str(i+1)]).append((locals()['ind_'+str(i+1)])[j])
    (locals()['ind'+str(i+1)]).append((locals()['ind_'+str(i+1)])[-1])
    if i!=0 and 2*chunk!=len((locals()['ind'+str(i+1)])):
        print("error: chunk size and interval limit does not match. flag number:",i)
    elif i==0 and 4*chunk!=len((locals()['ind'+str(i+1)])):
        print("error: chunk size and interval limit does not match. flag number: ",i)
    for k in range(channel_num):
        for g in range(int((len((locals()['ind'+str(i+1)])))/2)):
            locals()['channel'+str(k+1)+'_'+str(i+1)+'_chunk'+str(g+1)]=locals()['channel'+str(k+1)][(locals()['ind'+str(i+1)])[2*g]:(locals()['ind'+str(i+1)])[2*g+1]+1]
        

In [ ]:
#Run this cell if doing FFT
#DSP for individual chunk###########FFT
total_chunks=0
row=''
#generate average amplitude of each channel for indivisual label in specific band range
#variable_channel#_label#_chunk#

for i in np.unique(y):
    i=int(i)
    for k in range(channel_num):
        for g in range(int((len((locals()['ind'+str(i+1)])))/2)):
            data=locals()['channel'+str(k+1)+'_'+str(i+1)+'_chunk'+str(g+1)]
            
################################dsp:fft##################################
            freq,ffty=fft(data,fs)
            freq_ind=np.where((freq>=band[0])&(freq<=band[1]))[0]
            desired_freq=freq[freq_ind]
            amp_of_desired_freq=ffty[freq_ind]
            average_amp=np.mean(amp_of_desired_freq)
            locals()['average_amp_'+str(k+1)+'_'+str(i+1)+'_'+str(g+1)]=average_amp

            total_chunks+=1
if total_chunks != _total_chunk: print('error: total chunks number does not match with the data acqusition')

###################FFT input file generation##############################
for i in np.unique(y):
    i=int(i)
    for g in range(int((len((locals()['ind'+str(i+1)])))/2)):
            for k in range(channel_num):
                row+= str(locals()['average_amp_'+str(k+1)+'_'+str(i+1)+'_'+str(g+1)])
                row+=','
            row+=str(i)
            row+='\n'
with open(savedsp,'w') as f:
    f.write(row)
dspdata = np.loadtxt(savedsp,delimiter = ',')
dspx = dspdata[:,:-1]
dspy = dspdata[:,-1]

In [ ]:
#Run this cell if doing CSP
#DSP for individual chunk###################CSP
x_2class=np.append(x[ind_0],x[ind_2],axis=0)
y_2class=np.append(y[ind_0],y[ind_2],axis=0)
x=x_2class
y=y_2class
total_chunks=0
#generate average amplitude of each channel for indivisual label in specific band range
#variable_channel#_label#_chunk#

for i in np.unique(y):
    i=int(i)
    for k in range(channel_num):
        for g in range(int((len((locals()['ind'+str(i+1)])))/2)):
            data=locals()['channel'+str(k+1)+'_'+str(i+1)+'_chunk'+str(g+1)]
            channel_filtered=bandpass(band[0],band[1],data,fs)
            locals()['bp_filtered'+str(k+1)+'_'+str(i+1)+'_'+str(g+1)]=channel_filtered
            total_chunks+=1
if total_chunks != _total_chunk: print('error: total chunks number does not match with the data acqusition')

    
##################CSP input matrix generation#################################
labels=[]
for i in np.unique(y): 
    i=int(i)
    for g in range(int((len((locals()['ind'+str(i+1)])))/2)):
        for k in range(channel_num):
            data=locals()['bp_filtered'+str(k+1)+'_'+str(i+1)+'_'+str(g+1)]
            if k==0:
                CSPmatrices_mergechn=[data]
            else:
                CSPmatrices_mergechn=np.append(CSPmatrices_mergechn,[data],axis=0)
        if g==0 and i==-1: CSPmatrices=CSPmatrices_mergechn.reshape(1,CSPmatrices_mergechn.shape[0],CSPmatrices_mergechn.shape[1])
        else: CSPmatrices=np.append(CSPmatrices,CSPmatrices_mergechn.reshape(1,CSPmatrices_mergechn.shape[0],CSPmatrices_mergechn.shape[1]),axis=0)
        if i!=1: l=1
        else: l=0
        labels=np.append(labels,[l]) #####label =l -> different moving vs not moving   ####if previous cell ran: label = i for left vs right movement


In [ ]:
#Run this cell if doing CNN
#DSP for individual chunk###################CNN
total_chunks=0
#generate average amplitude of each channel for indivisual label in specific band range
#variable_channel#_label#_chunk#

for i in np.unique(y):
    i=int(i)
    for k in range(channel_num):
        for g in range(int((len((locals()['ind'+str(i+1)])))/2)):
            data=locals()['channel'+str(k+1)+'_'+str(i+1)+'_chunk'+str(g+1)]
            channel_filtered=bandpass(band[0],band[1],data,fs)
            locals()['bp_filtered'+str(k+1)+'_'+str(i+1)+'_'+str(g+1)]=channel_filtered
            total_chunks+=1
if total_chunks != _total_chunk: print('error: total chunks number does not match with the data acqusition')

    
##################CNN input matrix generation#################################
labels=[]
for i in np.unique(y): 
    i=int(i)
    for g in range(int((len((locals()['ind'+str(i+1)])))/2)):
        for k in range(channel_num):
            data=locals()['bp_filtered'+str(k+1)+'_'+str(i+1)+'_'+str(g+1)]
            if k==0:
                CNNmatrices_mergechn=[data]
            else:
                CNNmatrices_mergechn=np.append(CNNmatrices_mergechn,[data],axis=0)
        if g==0 and i==-1: CNNmatrices=CNNmatrices_mergechn.reshape(1,CNNmatrices_mergechn.shape[0],CNNmatrices_mergechn.shape[1])
        else: CNNmatrices=np.append(CNNmatrices,CNNmatrices_mergechn.reshape(1,CNNmatrices_mergechn.shape[0],CNNmatrices_mergechn.shape[1]),axis=0)
        labels=np.append(labels,[i])
_shape=CNNmatrices.shape
CNN_input=CNNmatrices.reshape(_shape[0],1,_shape[1],_shape[2])
lb = LabelBinarizer()
label_enc=lb.fit_transform(labels)
np.savetxt("labels.csv", label_enc, delimiter=",")

In [228]:
remove_list=[0]
drop_list=np.isin(train_label,remove_list,invert=True)
drop_array=np.array(drop_list)
label_keep=train_label[drop_array]
train_keep=train_data[drop_array]
# drop_list2=np.isin(test_labels,remove_list,invert=True)
# drop_array2=np.array(drop_list2)
# eval_keep=test_data[drop_array2]
# eval_label_keep=test_labels[drop_array2]

In [229]:
scaler=MinMaxScaler()
scaled_train_data=scaler.fit_transform(train_data)
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(train_keep,label_keep,test_size=0.1)
print(x_train.shape,y_train.shape)

((4050, 16), (4050,))


In [230]:
#Linear SVM classifier
clf1 = LinearSVC(C = 0.1, intercept_scaling=1, loss='hinge', max_iter=1000,multi_class='ovr', penalty='l2', random_state=1, tol=0.00001)
clf1.fit(x_train,y_train)
score_train1=clf1.score(x_train,y_train)
score_test1=clf1.score(x_test,y_test)
print(score_train1,score_test1)

(0.5, 0.5288888888888889)


In [231]:
model=KNeighborsClassifier(n_neighbors=7)
model.fit(train_data,train_label)
score_train=model.score(train_data,train_label)
score_test=model.score(test_data,test_label)
print(score_train,score_test)
model_filename='mindwave_model.pkl'
with open(model_filename,'wb') as file:
    pickle.dump(model,file)

(0.6877037037037037, 0.3482222222222222)


In [232]:
model=GradientBoostingClassifier()
model.fit(x_train,y_train)
score_train=model.score(x_train,y_train)
score_test=model.score(x_test,y_test)
print(score_train,score_test)
model_filename='mindwave_model.pkl'
with open(model_filename,'wb') as file:
    pickle.dump(model,file)

(0.7237037037037037, 0.62)


In [233]:
model=AdaBoostClassifier(n_estimators=100)#OneVsRestClassifier(SVC(kernel='linear'))
model.fit(x_train,y_train)
score_train=model.score(x_train,y_train)
score_test=model.score(x_test,y_test)
print(score_train,score_test)
# model_filename='mindwave_model.pkl'
# with open(model_filename,'wb') as file:
#     pickle.dump(model,file)

(0.6861728395061728, 0.5844444444444444)


In [234]:
model=DecisionTreeClassifier()
model.fit(x_train,y_train)
score_train=model.score(x_train,y_train)
score_test=model.score(x_test,y_test)
print(score_train,score_test)
model_filename='mindwave_model.pkl'
with open(model_filename,'wb') as file:
    pickle.dump(model,file,protocol=2)

(1.0, 0.6177777777777778)


In [235]:
model=LinearDiscriminantAnalysis()
model.fit(x_train,y_train)
score_train=model.score(x_train,y_train)
score_test=model.score(x_test,y_test)
print(score_train,score_test)
# model_filename='mindwave_model.pkl'
# with open(model_filename,'wb') as file:
#     pickle.dump(model,file)

(0.5997530864197531, 0.6222222222222222)


In [236]:
model2 = SVC(20,'rbf', degree=10, gamma='auto', coef0=0.0, tol=0.001, cache_size=10000, max_iter=-1, decision_function_shape='ovr')
model2.fit(x_train,y_train)
score_train2=model2.score(x_train,y_train)
score_test2=model2.score(x_test,y_test)
print(score_train2,score_test2)

(1.0, 0.47333333333333333)


In [242]:
=np.array([1,2,3])

In [289]:
rawdata=[]

In [250]:
rawdata.append(a)

In [294]:
rawdata

[array([2, 3, 4, 5, 6, 7, 8, 9]),
 array([2, 3, 4, 5, 6, 7, 8, 9]),
 array([2, 3, 4, 5, 6, 7, 8, 9]),
 array([2, 3, 4, 5, 6, 7, 8, 9])]

In [297]:
np.array(rawdata)

(4, 8)

In [258]:
a.reshape(1,3)

array([[1, 2, 3]])

In [260]:
sample_channel_data=[1,2,3,4,5,6,7,8]

In [261]:
csd_input=np.array(sample_channel_data).reshape(-1,1)

In [263]:
csd_input.shape

(8, 1)

In [264]:
csd_output=csd_input+1

In [287]:
out=csd_output.reshape(8,)

In [293]:
rawdata.append(out)

In [288]:
out

array([2, 3, 4, 5, 6, 7, 8, 9])